In [1]:
%matplotlib widget
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# ax1.spines['top'].set_visible(False) 

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]

saveFigures is set to: True
Done loading packages


In [2]:
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"

path_figs = os.getcwd() +"\\..\\Figures\\" 

In [3]:
# Data is (only) in the file from the most recent tuesday. 
# Should be made smarter, but here hardcoded
# tuePath = 'SSI_data_2022-01-04'
tuePath = 'SSI_data_2022-01-11'

dirPath = path_data + tuePath + '\\'

df1 = pd.read_csv(dirPath+'gennembrudsinfektioner_table1.csv',delimiter=';')

df2_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_cases.csv',delimiter=';')
df2_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_indlagte.csv',delimiter=';')
df2_D = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_dode.csv',delimiter=';')
df2_R = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_repositive.csv',delimiter=';')
df2_Int = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_antal_intensiv.csv',delimiter=';')

df3 = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_alle.csv',delimiter=';',decimal=",")
df3_C = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_cases.csv',delimiter=';',decimal=",")
df3_H = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_indlagte.csv',delimiter=';',decimal=",")
df3_D = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_dode.csv',delimiter=';',decimal=",")
df3_Int = pd.read_csv(dirPath+'gennembrudsinfektioner_table2_incidence_intensiv.csv',delimiter=';',decimal=",")

In [4]:
# Since order seems to be semi-random, we first need the correct order...
# weekNames = df.År.astype(str)+'-W'+df.Uge.apply(lambda x: f"{int(x):02d}")

import datetime
weekDTs = [np.datetime64(datetime.datetime.strptime(d[-4:] + '-W'+d[4:6]+'-1', "%Y-W%W-%w")) for d in df1.Ugenummer]

curOrder = np.argsort(weekDTs)
curOrder

array([14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0, 15],
      dtype=int64)

In [5]:
# df3_C
# df2_D
        
sNone = 'Ingen vaccination'        
sOne = 'Første vaccination'  
sTwo = 'Anden vaccination'
sFull = 'Fuld effekt efter primært forløb'   
sReva = 'Fuld effekt efter revaccination'


ageGroups = df2_C.Aldersgruppe.values
print(ageGroups)
# weekNames = df1.Ugenummer
weekNames = df1.Ugenummer.values[curOrder]
weekNamesShort = [x[4:6] for x in weekNames]
wInt = [int(x[4:6]) for x in weekNames]
wIntRange = np.arange(len(wInt))

allDates = np.array(weekDTs)[curOrder]
print(weekNames)

['0-5' '6-11' '12-15' '16-19' '20-29' '30-39' '40-49' '50-59' '60-64'
 '65-69' '70-79' '80+' '12+' 'Alle']
['Uge 38-2021' 'Uge 39-2021' 'Uge 40-2021' 'Uge 41-2021' 'Uge 42-2021'
 'Uge 43-2021' 'Uge 44-2021' 'Uge 45-2021' 'Uge 46-2021' 'Uge 47-2021'
 'Uge 48-2021' 'Uge 49-2021' 'Uge 50-2021' 'Uge 51-2021' 'Uge 52-2021'
 'Uge 01-2022']


In [6]:
# df1.iloc[:,40:50]

In [7]:
# Make function for gettings particular parts
def getTimeSeries(thisdf=df2_C,curStatus='Ingen vaccination',curAge='Alle',weekNames=weekNames):
    
    agedf = thisdf[thisdf.Aldersgruppe==curAge]
    allVals = []
    for curWeek in weekNames:
        toAdd = agedf[curWeek+'_'+curStatus].values[0]
        allVals.append(toAdd)
    allVals = np.array(allVals)

    return allVals

print(getTimeSeries(df2_H,sNone))
print(getTimeSeries(df2_H,sFull))

[ 32  38  53  73  90 113 160 145 210 220 222 241 282 315 356 346]
[ 54  42  54  75 136 167 192 269 298 302 320 349 369 356 457 392]


In [8]:
# Make a testplot of cases
allStatus = [sNone,sOne,sFull,sReva] # Omit "Two, but not full effect"
# allStatus = [sNone,sOne,sTwo,sFull,sReva] # Everything
cmap = plt.cm.get_cmap('Dark2',len(allStatus))
import matplotlib.colors
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","violet","blue"],len(allStatus))
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["purple","xkcd:mauve","green"],len(allStatus))
meanWidth = 5

# Set what to plot
curdf = df2_C
sYlabel = 'Tilfælde, antal'
# curdf = df2_D 
# sYlabel = 'Dødsfald, antal'
curdf = df3_C
sYlabel = 'Tilfælde, per 100.000'
# curdf = df3_D 
# sYlabel = 'Dødsfald per 100.000'
# curdf = df3_H 
# sYlabel = 'Indlæggelser per 100.000'
# curdf = df3_Int
# sYlabel = 'Intensiv, per 100.000'
# curAge = ageGroups[-4]
curAge = ageGroups[4]


fig,ax1 = plt.subplots()

ax1.set_title(curAge)
# for curStatus in allStatus:
for i in range(len(allStatus)):
    curStatus = allStatus[i]
    curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

    # ax1.plot(wInt,curTimeSeries,label=curStatus)
    
    # ax1.plot(wInt,curTimeSeries,'.:',linewidth=1,label=curStatus,color=cmap(i))
    # ax1.plot(rnTime(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=f'{meanWidth} ugers gennemsnit',color=cmap(i))

    # ax1.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
    ax1.plot(rnTime2(wIntRange,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

ax1.set_ylabel(sYlabel)


ax1.set_xticks(wIntRange)
ax1.set_xticklabels(weekNamesShort)

ax1.legend(loc='upper left')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# Make a testplot of cases
allStatus = [sNone,sOne,sFull,sReva] # Omit "Two, but not full effect"
# allStatus = [sNone,sOne,sTwo,sFull,sReva] # Everything
cmap = plt.cm.get_cmap('Dark2',len(allStatus))
import matplotlib.colors
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","violet","blue"],len(allStatus))
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["purple","xkcd:mauve","green"],len(allStatus))
meanWidth = 5

# Set what to plot
curdf = df2_C
sYlabel = 'Tilfælde, antal'
# curdf = df2_D 
# sYlabel = 'Dødsfald, antal'
curdf = df3_C
sYlabel = 'Tilfælde, per 100.000'
# curdf = df3_D 
# sYlabel = 'Dødsfald per 100.000'
# curdf = df3_H 
# sYlabel = 'Indlæggelser per 100.000'
# curdf = df3_Int
# sYlabel = 'Intensiv, per 100.000'
# curAge = ageGroups[-4]
curAge = ageGroups[4]


fig,ax1 = plt.subplots()

ax1.set_title(curAge)
# for curStatus in allStatus:
for i in range(len(allStatus)):
    curStatus = allStatus[i]
    curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

    # ax1.plot(wInt,curTimeSeries,label=curStatus)
    
    # ax1.plot(wInt,curTimeSeries,'.:',linewidth=1,label=curStatus,color=cmap(i))
    # ax1.plot(rnTime(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=f'{meanWidth} ugers gennemsnit',color=cmap(i))

    # ax1.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
    ax1.plot(rnTime2(allDates,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

ax1.set_ylabel(sYlabel)


# ax1.set_xticks(wIntRange)
# ax1.set_xticklabels(weekNamesShort)

ax1.legend(loc='upper left')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:

# Set what to plot
curAge = ageGroups[-3]
for curAge in ageGroups:
    fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,tight_layout=True)

    curdf = df2_C
    sYlabel = 'Antal'

    ax = ax1 
    ax.set_title('Smittetilfælde, '+curAge)
    # for curStatus in allStatus:
    for i in range(len(allStatus)):
        curStatus = allStatus[i]
        curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

        # ax.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
        ax.plot(rnTime2(wIntRange,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
    # sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

    ax.set_ylabel(sYlabel)
    ax.legend(loc='upper left')
    ax.set_ylim(bottom=0)
    ax.spines['top'].set_visible(False) 
    ax.spines['right'].set_visible(False)
    ax.grid(axis='y')


    ax.set_xticks(wIntRange)
    ax.set_xticklabels(weekNamesShort)

    curdf = df3_C
    sYlabel = 'Incidens per 100.000'
    ax = ax2
    for i in range(len(allStatus)):
        curStatus = allStatus[i]
        curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

        # ax.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
        ax.plot(rnTime2(wIntRange,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
    # sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

    ax.set_ylabel(sYlabel)
    # ax.legend(loc='upper left')
    ax.set_ylim(bottom=0)
    ax.set_xticks(wIntRange)
    ax.set_xticklabels(weekNamesShort)
    ax.spines['top'].set_visible(False) 
    ax.spines['right'].set_visible(False)
    ax.grid(axis='y')

    ax.set_xlim(left=wIntRange[meanWidth])
    ax.set_xlabel('Ugenummer')

    if saveFigures:
        fig.savefig(path_figs+'Breakthrough/CasesIncidence'+curAge)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:


# Set what to plot
# curAge = ageGroups[-3]
curAge = ageGroups[4]
for curAge in ageGroups:

    fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,tight_layout=True)

    curdf = df2_C
    sYlabel = 'Antal'

    ax = ax1 
    ax.set_title('Smittetilfælde, '+curAge)


    curVals = np.zeros(rnMean(curTimeSeries,meanWidth).shape)
    preVals = np.zeros(rnMean(curTimeSeries,meanWidth).shape)
    for i in range(len(allStatus)):
        curStatus = allStatus[i]
        curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

        curVals = curVals + rnMean(curTimeSeries,meanWidth)
        # ax.plot(rnTime2(wIntRange,meanWidth),curVals,label=curStatus,color=cmap(i))
        ax.fill_between(rnTime2(wIntRange,meanWidth),curVals,preVals,label=curStatus,color=cmap(i))
        preVals = curVals
        

    ax.set_ylabel(sYlabel)
    ax.legend(loc='upper left')
    ax.set_ylim(bottom=0)
    ax.spines['top'].set_visible(False) 
    ax.spines['right'].set_visible(False)
    ax.grid(axis='y')


    ax.set_xticks(wIntRange)
    ax.set_xticklabels(weekNamesShort)


    curdf = df2_C
    sYlabel = 'Andel [%]'
    ax = ax2
    totCase = curVals
    totCase = curVals/100 # For % 
    curVals = np.zeros(rnMean(curTimeSeries,meanWidth).shape)
    preVals = np.zeros(rnMean(curTimeSeries,meanWidth).shape)
    for i in range(len(allStatus)):
        curStatus = allStatus[i]
        curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

        curVals = curVals + rnMean(curTimeSeries,meanWidth)
        # ax.plot(rnTime2(wIntRange,meanWidth),curVals,label=curStatus,color=cmap(i))
        ax.fill_between(rnTime2(wIntRange,meanWidth),curVals/totCase,preVals/totCase,label=curStatus,color=cmap(i))
        preVals = curVals

    ax.set_ylabel(sYlabel)
    # ax.legend(loc='upper left')
    ax.set_ylim(bottom=0)
    ax.set_ylim(top=100)
    ax.set_xticks(wIntRange)
    ax.set_xticklabels(weekNamesShort)
    ax.spines['top'].set_visible(False) 
    ax.spines['right'].set_visible(False)
    ax.grid(axis='y')

    ax.set_xlim(left=wIntRange[meanWidth])
    ax.set_xlabel('Ugenummer')

    if saveFigures:
        fig.savefig(path_figs+'Breakthrough/CasesStacked'+curAge)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-11-70da4a5b7095>:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,tight_layout=True)


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:


# Set what to plot
curAge = ageGroups[-3]
# curAge = ageGroups[4]
for curAge in ageGroups:
    fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,tight_layout=True) 
    curdf = df2_H
    sYlabel = 'Antal'

    ax = ax1 
    ax.set_title('Indlæggelser, '+curAge+f' årige, {meanWidth} ugers gennemsnit')
    # for curStatus in allStatus:
    for i in range(len(allStatus)):
        curStatus = allStatus[i]
        curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

        # ax.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
        ax.plot(rnTime2(wIntRange,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
    # sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

    ax.set_ylabel(sYlabel)
    ax.legend(loc='upper left')
    ax.set_ylim(bottom=0)
    ax.set_xticks(wIntRange)
    ax.set_xticklabels(weekNamesShort)



    curdf = df3_H
    sYlabel = 'Per 100.000'
    ax = ax2

    curVals = np.zeros(rnMean(curTimeSeries,meanWidth).shape)
    preVals = np.zeros(rnMean(curTimeSeries,meanWidth).shape)
    for i in range(len(allStatus)):
        curStatus = allStatus[i]
        curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

        # curVals = curVals + rnMean(curTimeSeries,meanWidth)
        curVals = rnMean(curTimeSeries,meanWidth)
        ax.plot(rnTime2(wIntRange,meanWidth),curVals,label=curStatus,color=cmap(i))
        # ax.fill_between(rnTime2(wIntRange,meanWidth),curVals,preVals,label=curStatus,color=cmap(i))
        # preVals = curVals

    # sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

    ax.set_ylabel(sYlabel)
    ax.legend(loc='upper left')
    ax.set_ylim(bottom=0)
    ax.set_xticks(wIntRange)
    ax.set_xticklabels(weekNamesShort)

    ax.set_xlim(left=wIntRange[meanWidth])
    ax.set_xlabel('Ugenummer')
    fig.show()

    if saveFigures:
        fig.savefig(path_figs+'Breakthrough/Admissions'+curAge)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:

# Set what to plot
# curAge = ageGroups[-3]
for curAge in ageGroups[-6:]:
    fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,tight_layout=True)

    curdf = df2_D
    sYlabel = 'Antal'

    ax = ax1 
    ax.set_title('Dødsfald, '+curAge+f' årige, {meanWidth} ugers gennemsnit')
    # for curStatus in allStatus:
    for i in range(len(allStatus)):
        curStatus = allStatus[i]
        curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

        # ax.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
        ax.plot(rnTime2(wIntRange,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
    # sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

    ax.set_ylabel(sYlabel)
    ax.legend(loc='upper left')
    ax.set_ylim(bottom=0)
    ax.set_xticks(wIntRange)
    ax.set_xticklabels(weekNamesShort)



    curdf = df3_D
    sYlabel = 'Per 100.000'
    ax = ax2
    for i in range(len(allStatus)):
        curStatus = allStatus[i]
        curTimeSeries= getTimeSeries(curdf,curStatus,curAge)

        # ax.plot(rnTime2(wInt,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
        ax.plot(rnTime2(wIntRange,meanWidth),rnMean(curTimeSeries,meanWidth),label=curStatus,color=cmap(i))
    # sYlabel = sYlabel + f' [{meanWidth} ugers gennemsnit]'

    ax.set_ylabel(sYlabel)
    ax.legend(loc='upper left')
    ax.set_ylim(bottom=0)
    ax.set_xticks(wIntRange)
    ax.set_xticklabels(weekNamesShort)

    ax.set_xlim(left=wIntRange[meanWidth])
    ax.set_xlabel('Ugenummer')

    if saveFigures:
        fig.savefig(path_figs+'Breakthrough/Deaths'+curAge)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:

df1.columns[0:10]

Index(['Ugenummer', 'antal_personer_alle_Ingen vaccination',
       'antal_personer_alle_Første vaccination',
       'antal_personer_alle_Anden vaccination',
       'antal_personer_alle_Fuld effekt efter primært forløb',
       'antal_personer_alle_Fuld effekt efter revaccination',
       'antal_personer_Ingen vaccination', 'antal_personer_Første vaccination',
       'antal_personer_Anden vaccination',
       'antal_personer_Fuld effekt efter primært forløb'],
      dtype='object')

# Age-split bargraph

In [15]:
df2_H.iloc[:-2,-5:]

,Uge 01-2022_Ingen vaccination,Uge 01-2022_Første vaccination,Uge 01-2022_Anden vaccination,Uge 01-2022_Fuld effekt efter primært forløb,Uge 01-2022_Fuld effekt efter revaccination
0,79,0,0,0,0
1,16,4,0,0,0
2,5,5,11,11,0
3,5,1,28,25,3
4,51,7,87,70,14
5,53,4,78,68,10
6,21,2,86,67,19
7,34,2,81,50,30
8,17,3,47,17,30
9,11,3,45,24,21


In [22]:
weekNamesSmart = [x[-4:]+'-W'+x[4:6] for x in weekNames]
weekToShow = weekNames[-1]

# maxHospCount = df2_H.iloc[:,1:].max().max()
# maxHospInci  = df3_H.iloc[:,1:].max().max()

for weekToShow in weekNames:
    fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,tight_layout=True)

    thisdf = df2_H.iloc[:-2]

    curAges = thisdf['Aldersgruppe']
    curUnva = thisdf[weekToShow+'_Ingen vaccination']
    curBoth = thisdf[weekToShow+'_Anden vaccination']
    # curBoth = thisdf[weekToShow+'_Fuld effekt efter primært forløb']
    # curBoth = thisdf[weekToShow+'_Fuld effekt efter revaccination']

    curX = np.arange(len(curAges))
    curW = 0.3

    ax1.bar(curX-curW/2,curUnva,width=curW,color=cmap(0),label='Uvaccinerede')
    ax1.bar(curX+curW/2,curBoth,width=curW,color=cmap(3),label='To vaccinedoser eller mere')

    thisdf = df3_H.iloc[:-2]
    curAges = thisdf['Aldersgruppe']
    curUnva = thisdf[weekToShow+'_Ingen vaccination']
    curBoth = thisdf[weekToShow+'_Anden vaccination']
    # curBoth = thisdf[weekToShow+'_Fuld effekt efter primært forløb']
    # curBoth = thisdf[weekToShow+'_Fuld effekt efter revaccination']
    ax2.bar(curX-curW/2,curUnva,width=curW,color=cmap(0))
    ax2.bar(curX+curW/2,curBoth,width=curW,color=cmap(3))

    ax1.spines['top'].set_visible(False) 
    ax1.spines['right'].set_visible(False)
    ax2.spines['top'].set_visible(False) 
    ax2.spines['right'].set_visible(False)

    ax1.set_axisbelow(True)
    ax1.grid(axis='y')
    ax2.set_axisbelow(True)
    ax2.grid(axis='y')

    ax1.set_ylabel('Antal')
    ax2.set_ylabel('Incidens per 100.000')

    ax1.set_title('Nyindlæggelser, '+weekToShow)
    ax1.legend(loc='upper left')

    ax1.set_xticks(curX)
    ax2.set_xticks(curX)
    ax2.set_xticklabels(curAges)

    # ax1.set_ylim(top=maxHospCount)
    # ax2.set_ylim(top=maxHospInci)

    curFileWeek = weekToShow[-4:]+'-W'+weekToShow[4:6] 
    if saveFigures:
        fig.savefig(path_figs+'Breakthrough/UgeOverblik'+curFileWeek)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
display(df2_H.iloc[:-2,-5:])
display(df3_H.iloc[:-2,-5:])
87+35
df3_H

,Uge 01-2022_Ingen vaccination,Uge 01-2022_Første vaccination,Uge 01-2022_Anden vaccination,Uge 01-2022_Fuld effekt efter primært forløb,Uge 01-2022_Fuld effekt efter revaccination
0,79,0,0,0,0
1,16,4,0,0,0
2,5,5,11,11,0
3,5,1,28,25,3
4,51,7,87,70,14
5,53,4,78,68,10
6,21,2,86,67,19
7,34,2,81,50,30
8,17,3,47,17,30
9,11,3,45,24,21


,Uge 01-2022_Ingen vaccination,Uge 01-2022_Første vaccination,Uge 01-2022_Anden vaccination,Uge 01-2022_Fuld effekt efter primært forløb,Uge 01-2022_Fuld effekt efter revaccination
0,22.4,0.0,0.0,0.0,0.0
1,10.4,3.4,0.0,0.0,0.0
2,12.8,43.0,5.4,5.6,0.0
3,22.6,23.6,13.1,12.0,75.5
4,45.7,41.0,15.6,14.0,26.7
5,55.1,28.0,14.8,15.0,14.3
6,36.0,30.9,14.4,21.0,6.8
7,78.7,47.7,11.7,19.3,6.9
8,120.7,229.7,15.0,19.9,13.2
9,106.0,301.8,15.2,52.2,8.4


,Aldersgruppe,Uge 52-2021_Ingen vaccination,Uge 52-2021_Første vaccination,Uge 52-2021_Anden vaccination,Uge 52-2021_Fuld effekt efter primært forløb,Uge 52-2021_Fuld effekt efter revaccination,Uge 51-2021_Ingen vaccination,Uge 51-2021_Første vaccination,Uge 51-2021_Anden vaccination,Uge 51-2021_Fuld effekt efter primært forløb,...,Uge 38-2021_Ingen vaccination,Uge 38-2021_Første vaccination,Uge 38-2021_Anden vaccination,Uge 38-2021_Fuld effekt efter primært forløb,Uge 38-2021_Fuld effekt efter revaccination,Uge 01-2022_Ingen vaccination,Uge 01-2022_Første vaccination,Uge 01-2022_Anden vaccination,Uge 01-2022_Fuld effekt efter primært forløb,Uge 01-2022_Fuld effekt efter revaccination
0,0-5,24.6,7.9,0.0,0.0,0.0,12.2,0.0,0.0,0.0,...,0.5,0.0,0.0,0.0,0.0,22.4,0.0,0.0,0.0,0.0
1,6-11,7.0,2.9,0.0,0.0,0.0,4.6,1.6,0.0,0.0,...,0.3,0.0,0.0,0.0,0.0,10.4,3.4,0.0,0.0,0.0
2,12-15,7.3,7.4,2.0,2.1,0.0,9.2,7.1,1.5,1.6,...,0.0,0.0,0.0,0.0,NaN,12.8,43.0,5.4,5.6,0.0
3,16-19,0.0,41.2,7.9,7.6,53.1,12.6,19.0,5.1,5.2,...,2.9,0.0,0.0,0.0,0.0,22.6,23.6,13.1,12.0,75.5
4,20-29,47.1,31.5,12.9,12.6,16.3,27.1,34.5,6.7,6.7,...,1.8,0.0,0.6,0.6,0.0,45.7,41.0,15.6,14.0,26.7
5,30-39,59.9,55.6,17.0,16.0,18.3,45.4,40.4,11.0,9.8,...,4.2,4.0,0.0,0.0,0.0,55.1,28.0,14.8,15.0,14.3
6,40-49,68.8,81.8,12.5,11.7,18.1,83.4,51.6,7.5,6.9,...,5.0,0.0,0.2,0.2,0.0,36.0,30.9,14.4,21.0,6.8
7,50-59,79.7,63.9,14.5,14.0,16.4,98.1,0.0,10.4,10.0,...,9.1,0.0,0.4,0.4,0.0,78.7,47.7,11.7,19.3,6.9
8,60-64,98.2,69.6,16.0,16.0,16.2,103.1,68.7,12.4,12.9,...,11.6,0.0,1.3,1.3,0.0,120.7,229.7,15.0,19.9,13.2
9,65-69,114.2,292.7,17.4,22.8,13.0,139.8,0.0,13.0,13.5,...,16.0,105.9,0.7,0.7,0.0,106.0,301.8,15.2,52.2,8.4


# Plot the unvaccinated wave and the vaccinated wave seperate

In [18]:

# curTimeSeries= getTimeSeries(curdf,curStatus,curAge)
# print(df1.columns)
fig,ax1 = plt.subplots()


toPlot = df1['antal_cases_Ingen vaccination'].values[curOrder]
toPlot = df1['antal_cases_Ingen vaccination'].values[curOrder]/df1['antal_personer_Ingen vaccination'].values[curOrder]

ax1.plot(allDates,toPlot,'k.:',linewidth=1)
ax1.plot(rnTime(allDates,5),rnMean(toPlot,5),'k')

toPlot = df1['antal_cases_Anden vaccination'].values[curOrder] 
toPlot = df1['antal_cases_Anden vaccination'].values[curOrder]/df1['antal_personer_Anden vaccination'].values[curOrder]

ax1.plot(allDates,toPlot,'g.:',linewidth=1)
ax1.plot(rnTime(allDates,5),rnMean(toPlot,5),'g')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Old tests below

In [19]:
# fig,ax1 = plt.subplots()


# # allStatus = [sNone,sOne,sTwo,sFull,sReva] # Everything
# # for curStatus in allStatus:
# #     ax1.plot(weekNamesShort,df1['antal_personer_alle_'+curStatus].values[::-1],label=curStatus)
# # ax1.plot(weekNamesShort,df1['antal_personer_alle_Ingen vaccination'].values[::-1])


# curStatus = allStatus[0]
# curVals = df1['antal_personer_alle_'+curStatus].values[::-1]
# ax1.fill_between(weekNamesShort,curVals,label=curStatus)

# curStatus = allStatus[1]
# prevVals = curVals 
# curVals = prevVals + df1['antal_personer_alle_'+curStatus].values[::-1]
# ax1.fill_between(weekNamesShort,curVals,prevVals,label=curStatus)

# curStatus = allStatus[2]
# prevVals = curVals 
# curVals = prevVals + df1['antal_personer_alle_'+curStatus].values[::-1]
# ax1.fill_between(weekNamesShort,curVals,prevVals,label=curStatus)

# curStatus = allStatus[3]
# # prevVals = curVals 
# curVals = prevVals + df1['antal_personer_alle_'+curStatus].values[::-1]
# ax1.fill_between(weekNamesShort,curVals,prevVals,label=curStatus)
# curStatus = allStatus[4]
# prevVals = curVals 
# curVals = prevVals + df1['antal_personer_alle_'+curStatus].values[::-1]
# ax1.fill_between(weekNamesShort,curVals,prevVals,label=curStatus)

# ax1.legend(loc='center left')
# ax1.set_ylim(bottom=0)

In [20]:
# thisdf = df2_C
# curStatus = 'Ingen vaccination'
# curAge = '0-5'

# weekNames = df1.Ugenummer
# agedf = thisdf[thisdf.Aldersgruppe==curAge]
# allVals = []
# for curWeek in weekNames:
#     toAdd = agedf[curWeek+'_'+curStatus].values[0]
#     allVals.append(toAdd)
# allVals = np.array(allVals)

# # for k in range(0,len(thisdf)):
# #     curRow = thisdf.loc[k]

In [21]:
# # Flip dataframes to be like df1
# casedf = pd.DataFrame()
# casedf_once = pd.DataFrame()
# casedf_both = pd.DataFrame()
# casedf_full = pd.DataFrame()
# casedf_reva = pd.DataFrame()
# casedf_none = pd.DataFrame()

# weekNames = df1.Ugenummer
# casedf['Ugenummer'] = weekNames
# casedf_none['Ugenummer'] = weekNames
# casedf_once['Ugenummer'] = weekNames
# casedf_both['Ugenummer'] = weekNames
# casedf_full['Ugenummer'] = weekNames
# casedf_reva['Ugenummer'] = weekNames

# for k in range(0,len(df2_C)):
#     curRow = df2_C.loc[k]
#     curAge = df2_C.loc[k,'Aldersgruppe']
    
#     curNone = []
#     curOnce = []
#     curBoth = []
#     curFull = []
#     curReva = []
#     for curW in weekNames:
#         # print(curW)
        
#         thisNone = curRow[curW+'_Ingen vaccination']
#         curNone.append(thisNone)
        
#         thisOnce = curRow[curW+'_Første vaccination']
#         curOnce.append(thisOnce)
        
#         thisBoth = curRow[curW+'_Anden vaccination']
#         curBoth.append(thisBoth)
        
#         thisFull = curRow[curW+'_Fuld effekt efter primært forløb']
#         curFull.append(thisFull)
        
#         thisReva = curRow[curW+'_Fuld effekt efter revaccination']
#         curReva.append(thisReva)

    
#     # Add to dataframe
#     casedf[curAge+'_Ingen vaccination'] = curNone
#     casedf[curAge+'_Første vaccination'] = curOnce
#     casedf[curAge+'_Anden vaccination'] = curBoth 
#     casedf[curAge+'_Fuld effekt efter primært forløb'] = curFull
#     casedf[curAge+'_Fuld effekt efter revaccination'] = curReva
    
#     # Add to smaller dataframes as well
#     casedf_none[curAge] = curNone
#     casedf_once[curAge] = curOnce
#     casedf_both[curAge] = curBoth 
#     casedf_full[curAge] = curFull
#     casedf_reva[curAge] = curReva
    
# casedf